# EDA

## Getting Data

In [2]:
# getting access to data_prep.py functions inside flood_prediction package
import sys
sys.path.append('../')
sys.path.append('../flood_prediction')
from flood_prediction.data_prep import *

In [3]:
df = get_data_and_targets()
df.head()

,date,T(degC),rain(mm),surf_press(hPa),wind_s(km/h),wind_dir(deg),soil_moist_0_to_7cm(m3),soil_moist_7_to_28cm(m3),radiation(W/m2),river_discharge(m3/s),target
0,1984-01-01 00:00:00,19.2,0.0,982.1,4.5,331.0,0.251,0.278,0.0,48.77,0
1,1984-01-01 01:00:00,19.1,0.0,981.9,4.5,331.0,0.251,0.278,0.0,48.77,0
2,1984-01-01 02:00:00,18.6,0.0,982.0,4.1,345.0,0.251,0.278,0.0,48.77,0
3,1984-01-01 03:00:00,18.2,0.0,981.9,3.6,6.0,0.251,0.278,0.0,48.77,0
4,1984-01-01 04:00:00,17.7,0.0,981.9,3.7,11.0,0.251,0.278,0.0,48.77,0


## Basic EDA

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 347685 entries, 0 to 347684
Data columns (total 11 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   date                      347685 non-null  datetime64[ns]
 1   T(degC)                   347685 non-null  float64       
 2   rain(mm)                  347685 non-null  float64       
 3   surf_press(hPa)           347685 non-null  float64       
 4   wind_s(km/h)              347685 non-null  float64       
 5   wind_dir(deg)             347685 non-null  float64       
 6   soil_moist_0_to_7cm(m3)   347685 non-null  float64       
 7   soil_moist_7_to_28cm(m3)  347685 non-null  float64       
 8   radiation(W/m2)           347685 non-null  float64       
 9   river_discharge(m3/s)     347685 non-null  float64       
 10  target                    347685 non-null  int64         
dtypes: datetime64[ns](1), float64(9), int64(1)
memory usage: 29.2 MB


In [5]:
round(df.describe().transpose(),2)

,count,mean,std,min,25%,50%,75%,max
T(degC),347685.0,16.13,6.87,-6.80,11.10,15.7,20.70,37.80
rain(mm),347685.0,0.05,0.33,0.00,0.00,0.0,0.00,9.60
surf_press(hPa),347685.0,983.10,3.09,965.10,981.00,982.8,984.90,998.40
wind_s(km/h),347685.0,5.87,3.77,0.00,3.00,4.9,8.10,31.10
wind_dir(deg),347685.0,205.25,91.21,1.00,129.00,225.0,275.00,360.00
soil_moist_0_to_7cm(m3),347685.0,0.34,0.09,0.25,0.25,0.3,0.42,0.52
soil_moist_7_to_28cm(m3),347685.0,0.34,0.08,0.28,0.28,0.3,0.42,0.52
radiation(W/m2),347685.0,236.09,327.05,0.00,0.00,10.0,446.00,1149.00
river_discharge(m3/s),347685.0,36.64,40.52,4.66,17.45,26.1,40.98,1010.58
target,347685.0,0.00,0.05,0.00,0.00,0.0,0.00,1.00


In [6]:
df.isnull().sum()

date                        0
T(degC)                     0
rain(mm)                    0
surf_press(hPa)             0
wind_s(km/h)                0
wind_dir(deg)               0
soil_moist_0_to_7cm(m3)     0
soil_moist_7_to_28cm(m3)    0
radiation(W/m2)             0
river_discharge(m3/s)       0
target                      0
dtype: int64

In [7]:
# counting amount of target = 1
df['target'].value_counts()

0    346749
1       936
Name: target, dtype: int64

In [8]:
# printing when target is 1
df[df['target']==1][['date','river_discharge(m3/s)','target']]

,date,river_discharge(m3/s),target
4416,1984-07-03 00:00:00,179.72,1
4417,1984-07-03 01:00:00,179.72,1
4418,1984-07-03 02:00:00,179.72,1
4419,1984-07-03 03:00:00,179.72,1
4420,1984-07-03 04:00:00,179.72,1
...,...,...,...
346075,2023-06-24 19:00:00,124.03,1
346076,2023-06-24 20:00:00,124.03,1
346077,2023-06-24 21:00:00,124.03,1
346078,2023-06-24 22:00:00,124.03,1


## Is this the correct threshold? Are we missing any disaster date?

In [9]:
# threshold for river discharge= 240 m3/s
# printing every time river discharge is above threshold from 2015 to 2023 and target is 0 (grabbing every 24 rows to make it only 1 per day)
df[(df['river_discharge(m3/s)']>240) & (df['target']==0)][['date','river_discharge(m3/s)','target']][::24].tail(40)

,date,river_discharge(m3/s),target
49488,1989-08-24,244.47,0
66144,1991-07-19,324.09,0
66168,1991-07-20,306.61,0
67560,1991-09-16,301.00,0
73632,1992-05-26,246.17,0
73872,1992-06-05,285.42,0
74352,1992-06-25,324.84,0
75936,1992-08-30,267.53,0
117552,1997-05-30,248.46,0
118128,1997-06-23,402.65,0
